## **Pyramid Representation**

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import random 

In [ ]:
def PyramidRepresentation(image, scale = 1.5, minSize = (224,224)): 

    yield image 

    while True: 

        w = int(image.shape[1]/scale)
        image = cv2.resize(image, dsize=(w,w))

        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]: 
            break 

        yield image

In [ ]:
img = cv2.imread("/Users/furkandurmus/Desktop/Computer Vision/Images/husky.jpg")
im = PyramidRepresentation(img, 1.5, (10,10))
for i, image in enumerate(im): 
    print(i)

    if i == 5: 
        plt.imshow(image), plt.axis("off")

## **Sliding Window**

In [ ]:
def SlidingWindow(image, step, ws): 
    
    for y in range(0, image.shape[0] - ws[0], step): 
        for x in range(0, image.shape[1] - ws[1], step): 

            yield(x, y, image[y: y +ws[1], x: x+ws[0]])

In [ ]:
img = cv2.imread("/Users/furkandurmus/Desktop/Computer Vision/Images/husky.jpg")
im = SlidingWindow(img, 50, (200,150))

for i, image in enumerate(im): 
    print(i)

    if i == 100: 
        print(image[0], image[1])
        plt.imshow(image[2])

## **Non Max Supression**

In [ ]:
def NonMaxSupression(boxes, probs = None, overlapThresh=0.3): 

    if len(boxes) == 0: 
        return []
    
    if boxes.dtype.kind == "i": 
        boxes = boxes.astype("float")

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]

    # Find Field
    field = (x2 - x1 + 1)*(y2 - y1 + 1)   

    idxs = y2

    # Probability Values
    idxs = np.argsort(idxs)
    
    pick = [] #Select Boxes

    while len(idxs) > 0: 

        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        # Min and Max Values 
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # Find Width and Height
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        # Overlap 
        overlap = (w * h) / field[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
        
    return boxes[pick].astype("int")



## **Selective Search**

In [ ]:
image = cv2.imread("/Users/furkandurmus/Desktop/Computer Vision/Images/pyramid.jpg")
image = cv2.resize(image, dsize = (600,600))
cv2.imshow("Image", image)

In [ ]:
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)
ss.switchToSelectiveSearchQuality()

print("-> start")

rects = ss.process()

output = image.copy()

for (x,y,w,h) in rects[:50]: 
    color = [random.randint(0,255) for j in range(0,3)]
    cv2.rectangle(output, (x,y), (x+w, y+h), color, 2)

cv2.imshow("Output",output)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


## **Region Proposal Object Detection**

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input 
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils

from nms import NonMaxSupression


In [ ]:
def SelectiveSearch(image): 
    print("ss")

    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)

    ss.switchToSelectiveSearchQuality()

    rects = ss.process()

    return rects[:1000]


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Model 

model = ResNet50(weights="imagenet")
image = cv2.imread("/Users/furkandurmus/Desktop/Computer Vision/Images/animals.jpg")
image = cv2.resize(image, dsize = (400,400))

(H,W) = image.shape[:2]

# Selective Search

rects = SelectiveSearch(image)

proposals = []
boxes = []

for (x,y,w,h) in rects: 

    if w / float(W) < 0.1 or h / float(H) < 0.1: 
        continue

    roi = image[y:y + h, x:x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (224, 224))

    roi = img_to_array(roi)
    roi = preprocess_input(roi)

    proposals.append(roi)
    boxes.append((x, y, w, h))

proposals = np.array(proposals)

In [31]:
# Predict 

print("Predict")

preds = model.predict(proposals)
preds = imagenet_utils.decode_predictions(preds, top=1)

labels = {}
min_conf = 0.8

for (i,p) in enumerate(preds): 

    (_, label, prob) = p[0]

    if prob > min_conf and i < len(boxes):
        (x, y, w, h) = boxes[i]
        box = (x, y, x + w, y + h)
        L = labels.get(label, [])
        L.append((box, prob))
        labels[label] = L

clone = image.copy()

for label in labels.keys(): 

    for (box, prob) in labels[label]: 

        boxes = np.array([p[0] for p in labels[label]])
        proba = np.array([p[1] for p in labels[label]])
        boxes = NonMaxSupression(boxes, proba)

    for (startX, startY, endX, endY) in boxes:
            cv2.rectangle(clone, (startX, startY), (endX, endY),(0, 0, 255), 2)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.putText(clone, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            
    cv2.imshow("After", clone)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break


